In [2]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

In [3]:
db = database('trfc.db')

In [4]:
app, rt = fast_app()

In [21]:
user_inputs = {
    'min_season': 1921,
    'max_season': 2024,
    'league_tiers': [2, 3, 4, 5],
    'inc_play_offs': 1,
    'generic_comps': ['Anglo-Italian Cup', "Associate Members' Cup", 'FA Cup', 'FA Trophy', "Full Members' Cup", 'League Cup'],
    'pens_as_draw': 0,
    'venues': ['H', 'A', 'N'],
    'min_games': 10,
}

In [22]:
min_season = user_inputs['min_season']
max_season = user_inputs['max_season']
league_tiers = user_inputs['league_tiers']
inc_play_offs = user_inputs['inc_play_offs']
pens_as_draw = user_inputs['pens_as_draw']
venues = user_inputs['venues']
min_games = user_inputs['min_games']

In [23]:
if inc_play_offs == 0:
    po_filter = 'AND COALESCE(c.is_playoff, 0) != 1'
else:
    po_filter = ''

In [24]:
venue_placeholders = ','.join(['?' for _ in venues])

In [25]:
tier_placeholders = ','.join(['?' for _ in user_inputs['league_tiers']]) if user_inputs['league_tiers'] else ''

comp_placeholders = ','.join(['?' for _ in user_inputs['generic_comps']]) if user_inputs['generic_comps'] else ''

In [26]:
tier_comp_filter = ''
if tier_placeholders or comp_placeholders:
    filters = []
    if tier_placeholders:
        filters.append(f'r.league_tier IN ({tier_placeholders})')
    if comp_placeholders:
        filters.append(f'r.generic_comp IN ({comp_placeholders})')
    tier_comp_filter = 'AND (' + ' OR '.join(filters) + ')'

In [27]:
pens_as_draw = 1

In [28]:
query = f'''
    SELECT
        r.season,
        SUM(r.goal_diff == 1) AS plus_one,
        SUM(r.goal_diff == 2) AS plus_two,
        SUM(r.goal_diff == 3) AS plus_three,
        SUM(r.goal_diff == 4) AS plus_four,
        SUM(r.goal_diff >= 5) AS plus_five_up,
        SUM(r.goal_diff == 0) AS zero,
        SUM(r.goal_diff == -1) AS minus_one,
        SUM(r.goal_diff == -2) AS minus_two,
        SUM(r.goal_diff == -3) AS minus_three,
        SUM(r.goal_diff == -4) AS minus_four,
        SUM(r.goal_diff <= -5) AS minus_five_up
    FROM results r
    LEFT JOIN cup_game_details c ON r.game_date = c.game_date
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN seasons s ON r.season = s.season
    WHERE s.ssn_start >= ?
        AND s.ssn_start <= ?
        AND r.venue IN ({venue_placeholders})
        {po_filter}
        {tier_comp_filter}
    GROUP BY r.season
    HAVING COUNT(*) >= ?
    ORDER BY r.season DESC
'''

params = [
    min_season, max_season,
    *venues
]

if user_inputs['league_tiers']:
    params.extend(user_inputs['league_tiers'])

if user_inputs['generic_comps']:
    params.extend(user_inputs['generic_comps'])

params.append(min_games)

results = db.execute(query, tuple(params))

df = pd.DataFrame(
    results.fetchall(),
    columns=[d[0] for d in results.description]
)

df.head(10)

,season,plus_one,plus_two,plus_three,plus_four,plus_five_up,zero,minus_one,minus_two,minus_three,minus_four,minus_five_up
0,2024/25,6,1,1,0,0,6,4,5,3,1,0
1,2023/24,9,4,2,2,0,8,16,9,2,0,0
2,2022/23,10,3,3,0,0,16,11,10,0,0,0
3,2021/22,14,8,2,1,0,13,9,4,0,2,0
4,2020/21,17,7,1,0,1,18,10,2,1,2,1
5,2019/20,10,1,0,1,1,10,7,8,5,0,2
6,2018/19,13,8,2,1,0,16,10,4,2,0,2
7,2017/18,9,12,3,3,0,11,9,2,2,0,1
8,2016/17,19,5,8,1,1,11,8,4,0,0,0
9,2015/16,15,5,1,1,0,13,10,3,1,0,0


In [12]:
pens_as_draw = 0

In [13]:
query = f'''
    SELECT
        r.opposition,
        COUNT(*) as P,
        COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) != 1 AND r.outcome = 'D' AND c.pens_outcome = 'W')) OR r.outcome = 'W')
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) as W,
        COUNT(
            CASE WHEN
                (? = 0 AND (r.outcome = 'D' AND c.pens_outcome IS NULL))
            OR 
                (? = 1 AND r.outcome = 'D')
            THEN 1 END) as D,
        COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) != 1 AND r.outcome = 'D' AND c.pens_outcome = 'L')) OR r.outcome = 'L')
            OR 
                (? = 1 AND r.outcome = 'L')
            THEN 1 END) as L,
        SUM(r.goals_for) as GF,
        SUM(r.goals_against) as GA,
        SUM(r.goals_for) - SUM(r.goals_against) as GD,
        ROUND(CAST(COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) != 1 AND r.outcome = 'D' AND c.pens_outcome = 'W')) OR r.outcome = 'W')
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) AS FLOAT) / COUNT(*) * 100, 1) as win_pc
    FROM results r
    LEFT JOIN cup_game_details c ON r.game_date = c.game_date
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN seasons s ON r.season = s.season
    WHERE s.ssn_start >= ?
        AND s.ssn_start <= ?
        AND r.venue IN ({venue_placeholders})
        {po_filter}
        {tier_comp_filter}
    GROUP BY r.opposition
    HAVING COUNT(*) >= ?
    ORDER BY P DESC
'''

params = [
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    min_season, max_season,
    *venues
]

if user_inputs['league_tiers']:
    params.extend(user_inputs['league_tiers'])

if user_inputs['generic_comps']:
    params.extend(user_inputs['generic_comps'])

params.append(min_games)

results = db.execute(query, tuple(params))

df = pd.DataFrame(
    results.fetchall(),
    columns=[d[0] for d in results.description]
)

df.query("opposition=='Wigan Athletic'")

,opposition,P,W,D,L,GF,GA,GD,win_pc
69,Wigan Athletic,24,7,8,9,26.0,27.0,-1.0,29.2
